In [9]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase
import time
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS

import matplotlib
import matplotlib as mpl

import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
dir = '/scratch/Research/gutzwiller_research/GutzTestFiles/'

# Molecule file
molfile = dir + 'ts-0000.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol.xyz'

T = 300.0 # Temperature
dt = 0.1
C = 0.1 # Optimization convergence
steps = 40000

ntdir = '/nh/nest/u/jsmith/Research/gutzwiller_research/DarwinSets/GutzTrain/train_test_U2/U2_co3/'
cns = ntdir + 'rX-6.0A_32-4.0A_a8-8.params'
sae = ntdir + 'sae.dat'
nnf = ntdir + 'train'
Nn = 5

In [3]:
# Load molecule
mol = read(molfile)
L = 14.140479202201726
mol.set_cell(([[L, 0, 0],
               [0, L, 0],
               [0, 0, L]]))

mol.set_pbc((True, True, True))

In [4]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

In [5]:
def erg_rsix (r1,r2):
    return 1.0/np.power(np.linalg.norm(r1-r2),6)

def frc_rsix (r1,r2):
    F = np.empty(3)
    inv = 6.0/np.power(np.linalg.norm(r1-r2),4)
    F[0] = inv * (r1[0]-r2[0])
    F[1] = inv * (r1[1]-r2[1])
    F[2] = inv * (r1[2]-r2[2])
    return F

mol.calc.add_pairwise(erg_rsix, frc_rsix)

Adding pairwise potential!


In [7]:
# Optimize molecule
#start_time = time.time()
#dyn = QuasiNewton(mol)
#dyn.run(fmax=C)
#print('[ANI Total time:', time.time() - start_time, 'seconds]')

In [10]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(mol, 0.1 * units.fs)  # 5 fs time step.

In [13]:
def printenergy(a=mol):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
dyn.attach(printenergy, interval=50)

In [14]:
dyn.run(5000)

Energy per atom: Epot = 85.649eV  Ekin = 142.414eV (T=1101768K)  Etot = 228.063eV
Energy per atom: Epot = 68.106eV  Ekin = 187.809eV (T=1452952K)  Etot = 255.915eV
Energy per atom: Epot = 85.218eV  Ekin = 196.289eV (T=1518563K)  Etot = 281.508eV
Energy per atom: Epot = 80.163eV  Ekin = 222.526eV (T=1721540K)  Etot = 302.689eV


KeyboardInterrupt: 